In [ ]:
#устанавливаем spacy
!pip install spacy

In [ ]:
#устанавливаем русскоязычный модуль 
!python -m spacy download ru_core_news_sm

In [2]:
import spacy 
from spacy.training import Example
import json
import random
import pandas as pd

In [ ]:
#загружаем тестовые данные 
df = pd.read_csv('./test_data.csv')

In [62]:
def load_json(file):
    '''
    функция загружает файл формата json, где хранятся обучающие данные (фразы менеджера с разметкой 
    именованых сущностей: приветствие, прощание, представление менеджером себя, название компании)

    file - путь к файлу *.json
    '''
    with open(file, "r", encoding = 'utf-8') as file:
        data = json.load(file)
        return data
    
    
def train(train_data, iterations):
    '''
    функция обучает модель на поиск именованных сущеностей: приветствие, прощание, 
    представление менеджером себя, название компании

    train_data - список формата [['фраза', {'сущность': (метки)}]]
    iterations - количество итераций обучения
    '''
    
    #загружаем наименьшую рускоязычную модель spacy
    nlp = spacy.load("ru_core_news_sm") 
    #берём модуль распознавания сущеностей
    ner = nlp.get_pipe("ner")
    #добавляем в этот модуль новые сущности: приветствие, прощание, представление себя
    ner.add_label('Greeting')
    ner.add_label('Goodbye') 
    ner.add_label('Present') 

    #все другие модули, кроме распознавания сущеностей, отключаем 
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        #создаём оптимизатор
        optimizer = nlp.begin_training()
        #обучаем модель в течении заданного кол. итераций
        for itn in range(iterations):
            print(f'iteration: {itn}')
            #случайным образом перемешиваем обучающие данные
            random.shuffle(train_data)
            losses = {}
            #берём текст и метки из обучающих данных 
            for text, annotations in train_data:
                #здесь меняем формат данных согласно требованиям новой версии spacy 3.0
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                #обновляем веса модели с drop out параметром 0.25
                nlp.update([example],
                          sgd = optimizer,
                          drop = 0.2,
                          losses = losses
                          )
                #print(losses)
    return (nlp)

In [63]:
def make_table(trained, df):
    '''
    функция перебирает все диалоги таблицы df и находит именнованные сущности: приветствие, прощание,
    представление менеджером себя, название компании. Имя менеджера извлекается из сущности "представление менеджером себя"
    при помощи анализа синтаксическим зависимостей

    trained - рускоязычная модель spacy обученная находить сущности: приветствие, прощание,
    представление менеджером себя, название компании.

    df - таблица исходных данных
    '''

    #заводим пустые списки под извлекаемую информацию: приветствие, прощание, предствление себя, название компании, имя менеджера
    greet = []
    goodbye = []
    present = []
    company = []
    mang_name = []

    #обрабатываем каждый диалог отдельно
    for dialog in range(max(df.dlg_id)):
        #берём текст менеджера в конкретном диалоге
        mang_text = df[(df["dlg_id"] == dialog) & (df["role"] == "manager")].text
        # заводим временные пустые строки (будем добавлять к ним найденные сущности)
        tmp_greet = ''
        tmp_goodbye = ''
        tmp_present = ''
        tmp_company = '' 

        #некоторые сущености встречаются несколько раз в течении диалога,
        # поэтому заведём вот такие флаги, которые будут отключать поиск определённой сущности,
        # если ранее она уже была найдена в диалоге
        greed_found = False
        goodbye_found = False
        present_found = False
        company_found = False

        #здесь проходим по каждой фразе в текущем диалоге
        for phrase in mang_text:
            #создаём конвейер spacy на текущей фразе 
            doc = trained(phrase)
            #проходим по каждому слову в конвйере и если это именнованая сущность и она
            # не встречалась ранее в других фразах, то добавляем её в соответствующую временную строку
            for token in doc:
                if token.ent_type != 0:
                    if token.ent_type_ == 'Greeting' and greed_found == False:
                      tmp_greet += token.text + ' '
                    elif token.ent_type_ == 'Goodbye' and goodbye_found == False:
                        tmp_goodbye += token.text + ' '
                    elif token.ent_type_ == 'Present' and present_found == False:
                        tmp_present += token.text + ' '
                    elif token.ent_type_ == 'ORG' and company_found == False:
                        tmp_company += token.text + ' '
            
            #если именнованая сущеность найдена, то соответствующий флаг меняем на True
            if tmp_greet != '':
                greed_found = True
            if tmp_goodbye != '':
                goodbye_found = True
            if tmp_present != '':
                present_found = True
            if tmp_company != '':
                company_found = True

        #строки с извлечёнными сущностями добавляем в соответствующие списки
        greet.append(tmp_greet)
        goodbye.append(tmp_goodbye)    
        present.append(tmp_present)
        company.append(tmp_company)

    #здесь проходим по всем сущностям "представление менеджером себя" из списка
    for i, pres in enumerate(present):
          #создаём конвейр каждой фразы из списка с сущностями "представление менеджером себя
          doc = trained(pres[:-1])
          #проверяем: если слово классифицировано как имя собственное "PROPN" или
          #его синтаксическая зависимость опеределена как номинальный субъект "nsubj", а часть речи как существительное "NOUN",
          #то добавляем это слово в качестве имени менеджера
          for token in doc:
              if (token.pos_ == "PROPN") or  (token.dep_ == "nsubj" and token.pos_ == "NOUN"):
                  mang_name.append(token.text)
                  break
          #если ничего не найдено, то добавляем пустую строку
          if len(mang_name) == i:
              mang_name.append('') 

    #список req_manager обозначает выполнение/невыполнение менеджером требований
    #если найдена сущность "приветствие" и сущеность "прощание", то добавляем 'выполнено', иначе 'не выполнено'
    req_manager = ['выполнено' if greet[i] != '' and goodbye[i] != '' else 'не выполнено' for i in range(len(greet))]

    #создаём таблицу со всеми найденными сущеностями и метке о выполнении менеджером требований
    fin_tbl = pd.DataFrame({'приветствие': greet, 'представление': present, 'имя менеджера': mang_name, 
                            'компания': company, 'прощание': goodbye, 'требование к менеджеру': req_manager})

    return fin_tbl

In [64]:
#заружаем обучающий набор данных и обучаем
TRAIN_DATA = load_json('./TRAIN_DATA.json')
trained = train(TRAIN_DATA, 30)

iteration: 0


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "спасибо вам всего хорошего." with entities "[[12, 25, 'Goodbye']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "Добрый день я кристина компания звезда" with entities "[[0, 11, 'Greeting'], [13, 22, 'Present'], [32, 38...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Som

iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
iteration: 10
iteration: 11
iteration: 12
iteration: 13
iteration: 14
iteration: 15
iteration: 16
iteration: 17
iteration: 18
iteration: 19
iteration: 20
iteration: 21
iteration: 22
iteration: 23
iteration: 24
iteration: 25
iteration: 26
iteration: 27
iteration: 28
iteration: 29


In [65]:
#создаём таблицу
make_table(trained, df)

,приветствие,представление,имя менеджера,компания,прощание,требование к менеджеру
0,здравствуйте,Меня зовут ангелина,ангелина,диджитал бизнес,Всего хорошего,выполнено
1,здравствуйте,Меня зовут ангелина,ангелина,диджитал бизнес,всего хорошего,выполнено
2,здравствуйте,Меня зовут ангелина,ангелина,диджитал бизнес,,не выполнено
3,добрый день,Добрый меня максим зовут,максим,китобизнес,да понедельника тогда всего доброго,выполнено
4,,,,,до свидания,не выполнено
